<a href="https://colab.research.google.com/github/lov435/SOEmotions/blob/main/BERT_on_SO_comments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###1. Install required dependencies

In [9]:
!pip install torch
!pip install transformers
!pip install pandas
!pip install scikit-learn

### 2. Import the necessary packages

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import torch

### 3. Load the dataset, extract the necessary fields and split it into train and test

In [11]:
# Load the dataset
url='https://drive.google.com/file/d/1Suw8zSMSDQrtFtjFNnCXIipnkYIFDjvR/view?usp=sharing'
file_id=url.split('/')[-2]
dwn_url='https://drive.google.com/uc?id=' + file_id
df = pd.read_csv(dwn_url)
df.dropna(subset=['CommentTextProc'], inplace=True)

# Extract the relevant columns
comments = df['CommentTextProc'].astype(str).tolist()
labels = df['Group'].astype('category').cat.codes.tolist()

# Split the dataset into training and testing sets
train_comments, test_comments, train_labels, test_labels = train_test_split(
    comments, labels, test_size=0.2, random_state=42
)

### 4. Load the BERT tokenizer and model. Tokenize comments.

In [12]:
# Load the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(df['Group'].unique()))

# Tokenize the comments
train_encodings = tokenizer(train_comments, truncation=True, padding=True)
test_encodings = tokenizer(test_comments, truncation=True, padding=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 5. Create a custom dataset.

In [13]:
# Create a custom dataset
class CommentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CommentDataset(train_encodings, train_labels)
test_dataset = CommentDataset(test_encodings, test_labels)

### 6. Train the model

In [14]:
# Train the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

optimizer = AdamW(model.parameters(), lr=1e-5)

for epoch in range(5):
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
100%|██████████| 365/365 [01:04<00:00,  5.69it/s]


### 7. Evaluate the model

In [15]:

# Evaluate the model
model.eval()
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

all_preds = []
with torch.no_grad():
    for batch in tqdm(test_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()
        all_preds.extend(preds)

100%|██████████| 92/92 [00:06<00:00, 14.25it/s]


### 8. Calculate and print the accuracy

In [16]:
# Calculate and print the accuracy
accuracy = accuracy_score(test_labels, all_preds)
print(f"Overall Accuracy: {accuracy * 100:.2f}%")

Overall Accuracy: 66.30%
